In [255]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.ensemble import BaggingRegressor
from sklearn.neural_network import MLPRegressor

#iris = datasets.load_iris()
#print(iris)

df = pd.read_csv("C:\Users\Professor\Downloads\precos_casa_california.csv")
df = df.sample(frac=1)
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
13836,-116.14,34.22,32.0,3298.0,1228.0,763.0,360.0,1.8750,47800.0,INLAND
9110,-118.54,34.38,18.0,2096.0,309.0,1044.0,328.0,6.8299,262100.0,<1H OCEAN
8841,-118.47,34.03,29.0,3287.0,882.0,1523.0,823.0,3.7381,290600.0,<1H OCEAN
8788,-118.41,34.07,52.0,1202.0,142.0,408.0,138.0,15.0001,500001.0,<1H OCEAN
2558,-124.16,40.80,52.0,2167.0,480.0,908.0,451.0,1.6111,74700.0,NEAR OCEAN
15145,-117.36,33.20,19.0,1926.0,557.0,1190.0,483.0,1.3269,166100.0,NEAR OCEAN
7656,-118.12,33.94,31.0,2210.0,519.0,1047.0,472.0,3.3292,271300.0,<1H OCEAN
16066,-121.29,37.98,49.0,2501.0,565.0,1171.0,550.0,2.5043,76700.0,INLAND
19843,-119.05,36.07,21.0,2472.0,523.0,1238.0,504.0,1.7756,62900.0,INLAND
14740,-117.05,32.61,21.0,6034.0,1205.0,3795.0,1146.0,3.2633,129700.0,NEAR OCEAN


In [258]:
df.replace(' ', np.nan, inplace=True)
df = df.dropna()
colunas = ['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']

for i in colunas:
    df[i] = pd.to_numeric(df[i])
    
df_sem_out = df.copy()

for i in colunas:
    df_sem_out= df_sem_out.loc[df_sem_out[i] < df_sem_out[i].quantile(0.95)]
    df_sem_out= df_sem_out.loc[df_sem_out[i] > df_sem_out[i].quantile(0.05)]
    
df_sem_out = df_sem_out.reset_index(drop = True)
df_sem_out.replace('NEAR BAY', 0, inplace=True)
df_sem_out.replace('<1H OCEAN', 1, inplace=True)
df_sem_out.replace('INLAND', 2, inplace=True)
df_sem_out.replace('NEAR OCEAN', 3, inplace=True)
df_sem_out.replace('ISLAND', 4, inplace=True)
min_max_scaler = preprocessing.MinMaxScaler()
df_sem_out = min_max_scaler.fit_transform(df_sem_out)
df_sem_out = pd.DataFrame(df_sem_out)
type(df_sem_out)

pandas.core.frame.DataFrame

In [259]:
dados = df_sem_out.loc[:, df_sem_out.columns != 8]
target = df_sem_out.loc[:, df_sem_out.columns == 8]

In [320]:
tamanho_df = len(df_sem_out)
dados_treino = dados[:int(tamanho_df*0.8)]
dados_validacao = dados[int(tamanho_df*0.8):]
target_treino = target[:int(tamanho_df*0.8)]
target_validacao = target[int(tamanho_df*0.8):]
dados_treino.replace('NEAR BAY', 0, inplace=True)
dados_treino.replace('<1H OCEAN', 1, inplace=True)
dados_treino.replace('INLAND', 2, inplace=True)
dados_treino.replace('NEAR OCEAN', 3, inplace=True)
dados_treino.replace('ISLAND', 4, inplace=True)
dados_validacao.replace('NEAR BAY', 0, inplace=True)
dados_validacao.replace('<1H OCEAN', 1, inplace=True)
dados_validacao.replace('INLAND', 2, inplace=True)
dados_validacao.replace('NEAR OCEAN', 3, inplace=True)
dados_validacao.replace('ISLAND', 4, inplace=True)
dados_treino

,0,1,2,3,4,5,6,7,9
0,0.592593,0.193823,0.214286,0.299037,0.148541,0.268590,0.196429,0.930139,0.333333
1,0.635802,0.146965,0.523810,0.322402,0.427056,0.270255,0.482143,0.304981,0.333333
2,0.309671,0.577210,0.952381,0.382046,0.488064,0.339068,0.636905,0.157669,0.666667
3,0.204733,0.614483,0.404762,0.311129,0.188329,0.219756,0.269841,0.427130,0.000000
4,0.255144,0.512247,0.785714,0.466489,0.503979,0.728635,0.696429,0.375020,0.333333
5,0.526749,0.170394,0.500000,0.290223,0.226790,0.519423,0.321429,0.363091,1.000000
6,0.313786,0.654952,0.119048,0.630457,0.583554,0.713097,0.805556,0.478508,0.666667
7,0.620370,0.159744,0.571429,0.159664,0.648541,0.806326,0.763889,0.064361,0.333333
8,0.476337,0.461129,0.142857,0.355811,0.221485,0.399556,0.311508,0.658357,0.666667
9,0.658436,0.115016,0.642857,0.282845,0.212202,0.160932,0.236111,0.612854,0.333333


In [454]:
lucao = BaggingRegressor(n_estimators=13)

In [455]:
lucao.fit(dados_treino, target_treino)

BaggingRegressor(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=13, n_jobs=1, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

In [456]:
predicao = lucao.predict(dados_treino)

In [457]:
#resultado = target_treino.reset_index(drop = True).copy()
#predicao_teste = [float(i) for i in predicao]
#resultado['predicao'] = pd.Series(predicao_teste)
#resultado['diferenca'] = resultado['median_house_value'] - resultado['predicao']
#resultado['porcentagem'] = (resultado['median_house_value'] / resultado['predicao'] - 1)*100

In [458]:
lucao.score(dados_treino, target_treino)

0.9567461972967289

In [459]:
lucao.score(dados_validacao, target_validacao)

0.7617664122127903